<a href="https://colab.research.google.com/github/RicardoHernandezRodriguez/Hackated/blob/main/hackatec2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
from datetime import datetime, timedelta
import pandas as pd
from bs4 import BeautifulSoup
import re
import time
from typing import Dict, List, Optional
import sqlite3

class BuscadorNoticias:
    def __init__(self, news_api_key: Optional[str] = None):
        self.news_api_key = news_api_key
        self.keywords_trump = [
            'trump', 'presidente estados unidos', 'donald trump',
            'política comercial', 'aranceles', 'comercio méxico',
            'usmca', 't-mec', 'nafta'
        ]
        self.keywords_economia = [
            'mercado', 'bolsa', 'peso mexicano', 'inflación',
            'inversión', 'economía méxico', 'remesas',
            'wall street', 'mercados emergentes', 'tipo de cambio'
        ]
        self.fuentes_mexico = [
            'el-universal-mx', 'milenio', 'excelsior',
            'el-financiero-mx', 'proceso'
        ]
        self.fuentes_usa = [
            'cnn', 'reuters', 'associated-press', 'bbc-news',
            'the-wall-street-journal', 'bloomberg'
        ]
        self.init_database()

    def init_database(self):
        """Inicializa la base de datos SQLite para almacenar noticias"""
        self.conn = sqlite3.connect('noticias_economia.db')
        cursor = self.conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS noticias (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                titulo TEXT,
                descripcion TEXT,
                url TEXT UNIQUE,
                fuente TEXT,
                fecha_publicacion TEXT,
                categoria TEXT,
                relevancia INTEGER,
                fecha_guardado TEXT
            )
        ''')
        self.conn.commit()

    def buscar_con_newsapi(self, dias_atras: int = 7) -> List[Dict]:
        """Busca noticias usando NewsAPI"""
        if not self.news_api_key:
            print("⚠️  NewsAPI key no proporcionada. Usando fuentes gratuitas.")
            return []

        url = "https://newsapi.org/v2/everything"
        fecha_desde = (datetime.now() - timedelta(days=dias_atras)).strftime('%Y-%m-%d')

        noticias = []

        # Búsqueda combinada de palabras clave
        query = '(trump OR "presidente estados unidos") AND (méxico OR mexico OR "mercado" OR "economía")'

        params = {
            'q': query,
            'from': fecha_desde,
            'sortBy': 'relevancy',
            'language': 'es,en',
            'apiKey': self.news_api_key,
            'pageSize': 50
        }

        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()

            for article in data.get('articles', []):
                if self._es_relevante(article['title'], article.get('description', '')):
                    noticia = {
                        'titulo': article['title'],
                        'descripcion': article.get('description', ''),
                        'url': article['url'],
                        'fuente': article['source']['name'],
                        'fecha_publicacion': article['publishedAt'],
                        'categoria': self._categorizar_noticia(article['title'], article.get('description', '')),
                        'relevancia': self._calcular_relevancia(article['title'], article.get('description', ''))
                    }
                    noticias.append(noticia)

            print(f"✅ NewsAPI: {len(noticias)} noticias encontradas")

        except Exception as e:
            print(f"❌ Error en NewsAPI: {e}")

        return noticias

    def scrape_el_universal(self) -> List[Dict]:
        """Scraping de El Universal (ejemplo)"""
        noticias = []
        try:
            url = "https://www.eluniversal.com.mx/cartera"
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
            }

            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.content, 'html.parser')

            # Buscar artículos (esto puede variar según la estructura del sitio)
            articulos = soup.find_all('article', class_='article-item')[:10]

            for articulo in articulos:
                titulo_elem = articulo.find('h2') or articulo.find('h3')
                if titulo_elem:
                    titulo = titulo_elem.get_text().strip()
                    if self._es_relevante(titulo, ''):
                        link_elem = articulo.find('a')
                        url_completa = link_elem['href'] if link_elem else ''

                        noticia = {
                            'titulo': titulo,
                            'descripcion': '',
                            'url': url_completa,
                            'fuente': 'El Universal',
                            'fecha_publicacion': datetime.now().isoformat(),
                            'categoria': self._categorizar_noticia(titulo, ''),
                            'relevancia': self._calcular_relevancia(titulo, '')
                        }
                        noticias.append(noticia)

            print(f"✅ El Universal: {len(noticias)} noticias encontradas")

        except Exception as e:
            print(f"❌ Error scraping El Universal: {e}")

        return noticias

    def buscar_google_news(self, query: str) -> List[Dict]:
        """Búsqueda básica en Google News (sin API)"""
        noticias = []
        try:
            # Usar RSS de Google News (método alternativo gratuito)
            url = f"https://news.google.com/rss/search?q={query}&hl=es&gl=MX&ceid=MX:es"

            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'xml')

            items = soup.find_all('item')[:20]

            for item in items:
                titulo = item.title.text if item.title else ''
                descripcion = item.description.text if item.description else ''

                if self._es_relevante(titulo, descripcion):
                    noticia = {
                        'titulo': titulo,
                        'descripcion': descripcion,
                        'url': item.link.text if item.link else '',
                        'fuente': 'Google News',
                        'fecha_publicacion': item.pubDate.text if item.pubDate else '',
                        'categoria': self._categorizar_noticia(titulo, descripcion),
                        'relevancia': self._calcular_relevancia(titulo, descripcion)
                    }
                    noticias.append(noticia)

            print(f"✅ Google News: {len(noticias)} noticias encontradas")

        except Exception as e:
            print(f"❌ Error en Google News: {e}")

        return noticias

    def _es_relevante(self, titulo: str, descripcion: str) -> bool:
        """Determina si una noticia es relevante basándose en palabras clave"""
        texto_completo = (titulo + ' ' + descripcion).lower()

        # Debe contener al menos una palabra clave de Trump/Política
        tiene_trump = any(keyword in texto_completo for keyword in self.keywords_trump)

        # Y al menos una palabra clave económica
        tiene_economia = any(keyword in texto_completo for keyword in self.keywords_economia)

        # O menciones específicas de México en contexto económico
        tiene_mexico_economia = ('méxico' in texto_completo or 'mexico' in texto_completo) and \
                               any(word in texto_completo for word in ['mercado', 'economía', 'comercio', 'peso'])

        return (tiene_trump and tiene_economia) or tiene_mexico_economia

    def _categorizar_noticia(self, titulo: str, descripcion: str) -> str:
        """Categoriza la noticia según su contenido"""
        texto = (titulo + ' ' + descripcion).lower()

        if any(word in texto for word in ['arancel', 'comercio', 'exportación', 'importación']):
            return 'Comercio Internacional'
        elif any(word in texto for word in ['bolsa', 'mercado', 'inversión', 'wall street']):
            return 'Mercados Financieros'
        elif any(word in texto for word in ['peso', 'tipo de cambio', 'dólar']):
            return 'Tipo de Cambio'
        elif any(word in texto for word in ['inflación', 'economía', 'pib']):
            return 'Economía General'
        else:
            return 'Política Económica'

    def _calcular_relevancia(self, titulo: str, descripcion: str) -> int:
        """Calcula relevancia de 1-10 basándose en palabras clave"""
        texto = (titulo + ' ' + descripcion).lower()
        puntuacion = 0

        # Palabras clave de alta relevancia
        palabras_alta = ['trump', 'aranceles', 'comercio méxico', 'peso mexicano', 'wall street']
        puntuacion += sum(2 for palabra in palabras_alta if palabra in texto)

        # Palabras clave de media relevancia
        palabras_media = ['mercado', 'economía', 'inversión', 'méxico']
        puntuacion += sum(1 for palabra in palabras_media if palabra in texto)

        return min(puntuacion, 10)

    def guardar_noticias(self, noticias: List[Dict]):
        """Guarda noticias en la base de datos"""
        cursor = self.conn.cursor()
        guardadas = 0

        for noticia in noticias:
            try:
                cursor.execute('''
                    INSERT OR IGNORE INTO noticias
                    (titulo, descripcion, url, fuente, fecha_publicacion, categoria, relevancia, fecha_guardado)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    noticia['titulo'],
                    noticia['descripcion'],
                    noticia['url'],
                    noticia['fuente'],
                    noticia['fecha_publicacion'],
                    noticia['categoria'],
                    noticia['relevancia'],
                    datetime.now().isoformat()
                ))
                if cursor.rowcount > 0:
                    guardadas += 1
            except Exception as e:
                print(f"Error guardando noticia: {e}")

        self.conn.commit()
        print(f"💾 {guardadas} noticias nuevas guardadas en la base de datos")

    def buscar_todas_las_fuentes(self, dias_atras: int = 7) -> List[Dict]:
        """Ejecuta búsqueda en todas las fuentes disponibles"""
        print("🔍 Iniciando búsqueda en múltiples fuentes...\n")

        todas_noticias = []

        # NewsAPI (si está disponible)
        noticias_newsapi = self.buscar_con_newsapi(dias_atras)
        todas_noticias.extend(noticias_newsapi)

        # Google News
        queries = [
            'Trump México economía',
            'aranceles México comercio',
            'mercado peso mexicano',
            'presidente Estados Unidos México'
        ]

        for query in queries:
            noticias_google = self.buscar_google_news(query)
            todas_noticias.extend(noticias_google)
            time.sleep(1)  # Pausa para evitar bloqueos

        # Scraping (ejemplo con El Universal)
        noticias_scraping = self.scrape_el_universal()
        todas_noticias.extend(noticias_scraping)

        # Eliminar duplicados basándose en URL
        noticias_unicas = []
        urls_vistas = set()

        for noticia in todas_noticias:
            if noticia['url'] not in urls_vistas:
                noticias_unicas.append(noticia)
                urls_vistas.add(noticia['url'])

        print(f"\n📊 Total de noticias únicas encontradas: {len(noticias_unicas)}")

        return noticias_unicas

    def generar_reporte(self, formato: str = 'excel') -> str:
        """Genera reporte de noticias almacenadas"""
        cursor = self.conn.cursor()
        cursor.execute('''
            SELECT titulo, descripcion, url, fuente, fecha_publicacion,
                   categoria, relevancia, fecha_guardado
            FROM noticias
            ORDER BY relevancia DESC, fecha_publicacion DESC
        ''')

        noticias = cursor.fetchall()

        if not noticias:
            return "No hay noticias para reportar"

        # Crear DataFrame
        df = pd.DataFrame(noticias, columns=[
            'Título', 'Descripción', 'URL', 'Fuente', 'Fecha Publicación',
            'Categoría', 'Relevancia', 'Fecha Guardado'
        ])

        # Generar archivo
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

        if formato == 'excel':
            filename = f'reporte_noticias_{timestamp}.xlsx'
            df.to_excel(filename, index=False)
        else:
            filename = f'reporte_noticias_{timestamp}.csv'
            df.to_csv(filename, index=False)

        print(f"📄 Reporte generado: {filename}")
        return filename

    def mostrar_resumen(self):
        """Muestra resumen de noticias por categoría"""
        cursor = self.conn.cursor()
        cursor.execute('''
            SELECT categoria, COUNT(*) as cantidad, AVG(relevancia) as relevancia_promedio
            FROM noticias
            GROUP BY categoria
            ORDER BY cantidad DESC
        ''')

        print("\n📈 RESUMEN POR CATEGORÍAS:")
        print("-" * 50)

        for row in cursor.fetchall():
            categoria, cantidad, relevancia = row
            print(f"{categoria}: {cantidad} noticias (Relevancia promedio: {relevancia:.1f})")

    def cerrar_conexion(self):
        """Cierra la conexión a la base de datos"""
        self.conn.close()

# Ejemplo de uso
if __name__ == "__main__":
    # Crear instancia del buscador
    # Para usar NewsAPI, proporciona tu API key: BuscadorNoticias(news_api_key="tu_api_key")
    buscador = BuscadorNoticias()

    try:
        # Buscar noticias de los últimos 7 días
        noticias = buscador.buscar_todas_las_fuentes(dias_atras=7)

        # Guardar en base de datos
        buscador.guardar_noticias(noticias)

        # Mostrar resumen
        buscador.mostrar_resumen()

        # Generar reporte
        buscador.generar_reporte('excel')

        # Mostrar las 5 noticias más relevantes
        print("\n🔥 TOP 5 NOTICIAS MÁS RELEVANTES:")
        print("-" * 50)

        noticias_ordenadas = sorted(noticias, key=lambda x: x['relevancia'], reverse=True)
        for i, noticia in enumerate(noticias_ordenadas[:5], 1):
            print(f"{i}. [{noticia['categoria']}] {noticia['titulo']}")
            print(f"   Fuente: {noticia['fuente']} | Relevancia: {noticia['relevancia']}/10")
            print(f"   URL: {noticia['url']}\n")

    finally:
        buscador.cerrar_conexion()

🔍 Iniciando búsqueda en múltiples fuentes...

⚠️  NewsAPI key no proporcionada. Usando fuentes gratuitas.
✅ Google News: 8 noticias encontradas
✅ Google News: 3 noticias encontradas
✅ Google News: 8 noticias encontradas
✅ Google News: 1 noticias encontradas
✅ El Universal: 0 noticias encontradas

📊 Total de noticias únicas encontradas: 19
💾 19 noticias nuevas guardadas en la base de datos

📈 RESUMEN POR CATEGORÍAS:
--------------------------------------------------
Comercio Internacional: 8 noticias (Relevancia promedio: 4.2)
Tipo de Cambio: 4 noticias (Relevancia promedio: 3.0)
Economía General: 4 noticias (Relevancia promedio: 3.8)
Mercados Financieros: 2 noticias (Relevancia promedio: 5.0)
Política Económica: 1 noticias (Relevancia promedio: 2.0)
📄 Reporte generado: reporte_noticias_20250526_153933.xlsx

🔥 TOP 5 NOTICIAS MÁS RELEVANTES:
--------------------------------------------------
1. [Comercio Internacional] Economía prevé un impacto limitado para México si Trump pone arancele